## Optimal Team Selection via the solution to the Knapsack Problem

In this notebook we construct an optimal team (that matches a given formation, i.e 4-4-4, 4 defensemen, 4 Midfielders and 4 fowards) from a set of players. The formation, along with the cost you pay for each position type is an adjustable parameter (for example, a general manager may wish to allocate less of his resources towards the Def position, opting to spend more on Fwds). Possible usecases include: setting lineups automatically in fantasy sports or for GMs (including those for popular Ultimate Team e-sport games) to optimize their lineups. 

In [19]:
import requests
import pandas as pd
import numpy as np
from dimod import BinaryQuadraticModel
from dwave.system import LeapHybridSampler

import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [3]:
slim_elements_df = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]
df = slim_elements_df.merge(elements_types_df, left_on = 'element_type', right_on = 'id')

df = df[['plural_name_short','second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

In [4]:
df = df[df.minutes > 100]
df.loc[df.plural_name_short == 'GKP', 'class'] = 1
df.loc[df.plural_name_short == 'DEF', 'class'] = 2
df.loc[df.plural_name_short == 'FWD', 'class'] = 3
df.loc[df.plural_name_short == 'MID', 'class'] = 4

df_mid = df[df.plural_name_short == 'MID']
df_gk = df[df.plural_name_short == 'GKP']
df_def = df[df.plural_name_short == 'DEF']
df_fwd = df[df.plural_name_short == 'FWD']

In [6]:
def set_up_bqm(df1, percent, max_weight):
    df1['value_season'] = df1['value_season'].astype(float)
    values = list(df1['value_season'])
    weights = list(df1['now_cost'])
    volumes = [x/x for x in range(1, len(values))]

    n = len(values)
    variables = list(range(n))
    weight = max_weight*percent

    return variables, values, volumes


def bqm_position(variables, values, volumes, max_volume):

    bqm = BinaryQuadraticModel('BINARY')

    variables = [bqm.add_variable(v, -values[v]) for v in variables]
    
    slacks_volume = bqm.add_linear_equality_constraint(
    [(x, v) for x, v in zip(variables, volumes)],
    constant=-max_volume,
    lagrange_multiplier=500
    )

    return bqm


def get_players(bqm, df):
    sampler = LeapHybridSampler(token = "KmJQ-eb7dea9880650063660800305a6750d9fe70bb21")
    response = sampler.sample(
        bqm, time_limit=25,
        )
    best_solution = response.first.sample
    indices = []

    for i, v in best_solution.items():
        if v != 0:
            indices.append(i)

    return pd.DataFrame(df.iloc[indices][['second_name','plural_name_short' ]])

In [20]:
variables_def, values_def, volumes_def = set_up_bqm(df_def, 20, 1000)
variables_mid, values_mid, volumes_mid = set_up_bqm(df_mid, 30, 1000)
variables_fwd, values_fwd, volumes_fwd = set_up_bqm(df_fwd, 30, 1000)
variables_gk, values_gk, volumes_gk = set_up_bqm(df_gk, 20, 1000)

bqm_mid=bqm_position(variables_mid,values_mid,volumes_mid, 3)
bqm_fwd=bqm_position(variables_fwd,values_fwd,volumes_fwd, 1)
bqm_gk=bqm_position(variables_gk,values_gk,volumes_gk, 0)
bqm_def=bqm_position(variables_def,values_def,volumes_def, 3)

In [18]:
df1 = get_players(bqm_gk, df_gk)
df2 = get_players(bqm_def, df_def)
df3 = get_players(bqm_mid , df_mid)
df4 = get_players(bqm_fwd , df_fwd)

pd.concat([df1, df2, df3, df4])

,second_name,plural_name_short
47,dos Santos Patrício,GKP
330,Targett,DEF
421,Wan-Bissaka,DEF
469,Cresswell,DEF
485,Ait Nouri,DEF
145,Dallas,MID
152,Harrison,MID
244,Soucek,MID
253,Otasowie,MID
285,Bamford,FWD
